In [15]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque
from datetime import datetime


class ViolenceDetectorTFLite:
    def __init__(self, model_path, fight_threshold=0.85):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_shape = self.input_details[0]['shape']
        self.frame_sequence = deque(maxlen=16)
        self.prediction_history = deque(maxlen=8)
        self.fight_threshold = fight_threshold

    def preprocess(self, frames):
        resized = [cv2.resize(f, (64, 64)) for f in frames]
        normalized = np.array(resized) / 255.0
        return normalized.astype(np.float32).reshape(1, 16, 64, 64, 3)

    def predict(self, frame):
        self.frame_sequence.append(frame)
        if len(self.frame_sequence) < 16:
            return "Safe"

        input_data = self.preprocess(list(self.frame_sequence))
        self.interpreter.set_tensor(self.input_details[0]['index'], input_data)
        self.interpreter.invoke()
        output_score = self.interpreter.get_tensor(self.output_details[0]['index'])[0][0]

        self.prediction_history.append(output_score)
        avg_score = sum(self.prediction_history) / len(self.prediction_history)

        if avg_score > self.fight_threshold:
            return "🚨 FIGHT DETECTED - TAKE ACTION"
        else:
            return "✅ Safe"


def main():
    model_path = "violence_model.tflite"
    detector = ViolenceDetectorTFLite(model_path)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: Webcam not accessible!")
        return

    print("✅ Webcam opened. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to grab frame.")
            break

        small_frame = cv2.resize(frame, (320, 240))

        # Predict violence label
        status = detector.predict(small_frame)
        color = (0, 0, 255) if "FIGHT" in status else (0, 255, 0)

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        overlay_text = f"{status} | {timestamp}"

        cv2.putText(frame, overlay_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), color, 3)

        cv2.imshow("📷 Real-Time Violence Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()





✅ Model loaded. Expected input: (None, 16, 64, 64, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
🧠 Result: ⚠️ Fight Detected!


In [1]:
import streamlit as st
import cv2
import numpy as np
import tempfile
from tensorflow.keras.models import load_model

# Load trained model once
@st.cache_resource
def load_violence_model():
    model = load_model("violence_detection_model (1).h5")
    return model

model = load_violence_model()

# Frame extraction function
def extract_frames_from_video(video_path, num_frames=16, target_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        return None

    frame_indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.resize(frame, target_size)
            frame = frame / 255.0
            frames.append(frame)

    cap.release()
    return np.array(frames)

# Prediction function
def predict_violence(frames, model):
    if frames.shape != (16, 64, 64, 3):
        return "❌ Frame input shape mismatch"
    input_data = np.expand_dims(frames, axis=0)
    prediction = model.predict(input_data)[0][0]
    return "⚠️ Fight Detected!" if prediction > 0.5 else "✅ No Fight Detected."

# Streamlit UI
st.set_page_config(page_title="Violence Detection", layout="centered")
st.title("🔍 Violence Detection in Videos")

video_file = st.file_uploader("Upload a Video", type=["mp4", "avi", "mov"])

if video_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_video:
        temp_video.write(video_file.read())
        video_path = temp_video.name

    st.video(video_file)
    st.write("⏳ Extracting frames and running prediction...")

    frames = extract_frames_from_video(video_path)

    if frames is None or len(frames) < 16:
        st.error("❌ Couldn't extract enough frames from the video.")
    else:
        result = predict_violence(frames, model)
        st.success(f"🧠 Prediction Result: {result}")


2025-06-19 15:46:43.350 
  command:

    streamlit run C:\Users\Kalindi Mishra\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
